In [1]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer

In [3]:
from pyspark import SparkContext

sc = SparkContext()
sqlContext = SQLContext(sc)
df = sqlContext.read.load('daily_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')
df.columns

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/02 15:21:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/02 15:21:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/soe/srmahesh/miniconda3/envs/sampleenv/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [4]:
featureColumns = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am',
        'max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am',
        'rain_duration_9am']

In [12]:
df[featureColumns].show(2, vertical=True)

-RECORD 0------------------------------------
 air_pressure_9am       | 918.0600000000087  
 air_temp_9am           | 74.82200000000041  
 avg_wind_direction_9am | 271.1              
 avg_wind_speed_9am     | 2.080354199999768  
 max_wind_direction_9am | 295.39999999999986 
 max_wind_speed_9am     | 2.863283199999908  
 rain_accumulation_9am  | 0.0                
 rain_duration_9am      | 0.0                
-RECORD 1------------------------------------
 air_pressure_9am       | 917.3476881177097  
 air_temp_9am           | 71.40384263106537  
 avg_wind_direction_9am | 101.93517935618371 
 avg_wind_speed_9am     | 2.4430092157340217 
 max_wind_direction_9am | 140.47154847112498 
 max_wind_speed_9am     | 3.5333236016106238 
 rain_accumulation_9am  | 0.0                
 rain_duration_9am      | 0.0                
only showing top 2 rows



In [37]:
!pip install pyarrow numpy

##### ANOTHER WAY

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [39]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [41]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [42]:
df

number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,918.0600000000087,74.82200000000041,271.1,2.080354199999768,295.39999999999986,2.863283199999908,0.0,0.0,42.42000000000046,36.160000000000494
1,917.3476881177097,71.40384263106537,101.93517935618371,2.4430092157340217,140.47154847112498,3.5333236016106238,0.0,0.0,24.328697291802207,19.4265967985621
2,923.0400000000084,60.637999999998776,51.000000000000036,17.067852199999727,63.6999999999999,22.100967200000003,0.0,20.0,8.900000000000047,14.460000000000045
3,920.5027511759195,70.13889486830053,198.83213271746368,4.337363056162703,211.20334124325618,5.190045360219378,0.0,0.0,12.189101868764444,12.742547353761848
4,921.1600000000036,44.29400000000284,277.8,1.8566601999998955,136.49999999999991,2.863283199999954,8.899999999998613,14730.0,92.41000000000044,76.74000000000046
5,915.300000000006,78.40400000000422,182.8,9.93201359999979,188.99999999999986,10.983375399999995,0.02000000000043656,170.0,35.130000000000265,33.930000000000256
6,915.5988675132801,70.04330431686239,177.87540718659943,3.7455865377244795,186.60669587586753,4.589632428699432,0.0,0.0,10.657421657822635,21.385656725200974
7,918.0700000000023,51.710000000002346,242.4,2.527742199999939,271.5999999999999,3.6462121999999924,0.0,0.0,80.47000000000041,74.92000000000041
8,920.080000000006,80.58200000000384,40.7,4.518618799999764,62.99999999999986,5.883152199999977,0.0,0.0,29.58000000000045,24.030000000000427
9,915.0100000000116,47.4979999999998,163.10000000000005,4.943637399999545,195.89999999999992,6.576603599999951,0.0,0.0,88.6000000000012,68.0500000000012


In [47]:
df_x = spark.read.csv('marketing_data.csv', header=True)

In [58]:
df_x

ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
1826,1970,Graduation,Divorced,"$84,835.00",0,0,6/16/14,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP
1,1961,Graduation,Single,"$57,091.00",0,0,6/15/14,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA
10476,1958,Graduation,Married,"$67,267.00",0,1,5/13/14,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US
1386,1967,Graduation,Together,"$32,474.00",1,1,5/11/14,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS
5371,1989,Graduation,Single,"$21,474.00",1,0,4/8/14,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP
7348,1958,PhD,Single,"$71,691.00",0,0,3/17/14,0,336,130,411,240,32,43,1,4,7,5,2,0,0,0,0,0,1,0,SP
4073,1954,2n Cycle,Married,"$63,564.00",0,0,1/29/14,0,769,80,252,15,34,65,1,10,10,7,6,1,0,0,0,0,1,0,GER
1991,1967,Graduation,Together,"$44,931.00",0,1,1/18/14,0,78,0,11,0,0,7,1,2,1,3,5,0,0,0,0,0,0,0,SP
4047,1954,PhD,Married,"$65,324.00",0,1,1/11/14,0,384,0,102,21,32,5,3,6,2,9,4,0,0,0,0,0,0,0,US
9477,1954,PhD,Married,"$65,324.00",0,1,1/11/14,0,384,0,102,21,32,5,3,6,2,9,4,0,0,0,0,0,0,0,IND


In [67]:
df_x.createTempView("tableB")

In [79]:
spark.sql("select DISTINCT * from tableB where Year_Birth> 1986 order by ID desc LIMIT 5")

ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
999,1991,Graduation,Single,"$86,037.00",0,0,1/2/13,95,490,44,125,29,20,22,1,6,7,11,3,0,1,1,0,0,1,0,CA
9960,1991,Basic,Single,"$26,868.00",1,0,4/30/14,52,0,0,1,8,3,2,1,1,0,2,7,0,0,0,0,0,0,0,SA
9909,1996,2n Cycle,Married,"$7,500.00",0,0,11/9/12,24,3,18,14,15,22,50,3,3,1,3,9,0,0,0,0,0,1,0,AUS
9771,1988,2n Cycle,Single,"$23,331.00",1,0,10/5/12,97,104,20,101,24,15,5,6,5,1,5,9,0,0,0,0,0,0,0,IND
9701,1988,Graduation,Together,"$41,883.00",1,0,3/19/13,13,32,34,41,73,16,116,3,4,2,3,7,0,0,0,0,0,0,0,CA


In [77]:
dfff = spark.sql("select DISTINCT * from tableB where Year_Birth > 1986 order by ID desc LIMIT 5")

In [82]:
dfff.write.csv("out.csv")

#### RDD

In [95]:
rdd = sc.parallelize([('C',85,76,87,91), ('B',85,76,87,91), ("A", 85,78,96,92), ("A", 92,76,89,96)], 4)

In [94]:
type(rdd)

pyspark.rdd.RDD

In [103]:
print(rdd)

ParallelCollectionRDD[306] at readRDDFromFile at PythonRDD.scala:274


In [107]:
df[df.air_temp_9am < 50]

number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
4,921.1600000000036,44.29400000000284,277.8,1.8566601999998955,136.49999999999991,2.863283199999954,8.899999999998613,14730.0,92.41000000000044,76.74000000000046
9,915.0100000000116,47.4979999999998,163.10000000000005,4.943637399999545,195.89999999999992,6.576603599999951,0.0,0.0,88.6000000000012,68.0500000000012
11,915.6400000000099,45.71599999999757,241.60000000000005,5.860782799999633,265.7999999999999,8.030614600000042,0.5499999999999545,1770.0,90.56000000000022,79.0900000000002
12,917.3900000000112,49.78399999999978,204.10000000000005,1.2750557999995473,211.79999999999993,2.01324599999997,0.0,0.0,73.1500000000012,58.43000000000119
23,920.3500000000046,47.5700000000002,192.10000000000005,6.2634319999996935,205.69999999999993,7.605596000000003,0.0,0.0,54.55,66.00000000000001
26,914.4900000000038,49.892000000005595,163.0,4.854159799999914,189.59999999999994,6.733189399999966,0.0,0.0,92.10000000000042,90.9900000000004
28,915.8400000000056,40.11800000000114,171.90000000000003,10.535987399999618,187.99999999999991,12.929513199999962,0.0,0.0,86.43000000000026,84.39000000000027
29,916.3100000000024,45.42800000000261,183.1,8.343786199999892,194.59999999999994,10.088599399999971,0.0,0.0,86.15000000000046,90.58000000000048
42,921.5000000000059,49.55000000000121,90.70000000000003,1.7448131999996142,112.29999999999993,2.684327999999957,1.5299999999997451,5420.0,34.35000000000028,49.50000000000024
55,916.420000000002,45.014000000005645,82.8,3.064607800000057,129.7999999999999,4.809421000000055,8.739999999999782,7000.0,57.710000000000136,41.80000000000011


In [108]:
pwd

'/soe/srmahesh/bigdata/big-data-4'

In [ ]:
!pyspark

Python 3.9.5 (default, Jun  4 2021, 12:28:51) 
[GCC 7.5.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/02 18:33:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/02 18:33:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/02 18:33:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.2.1
      /_/

Using Python version 3.9.5 (default, Jun  4 2021 12:28:51)
Spark context Web UI available at http://nlp-gpu-01.soe.ucsc.edu:4042
Spark contex